In [1]:
# загрузим данные
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import preprocessing

# Загрузка данных
df = pd.read_csv('railway_freight_data.csv')

# Преобразование категориальных данных в числовые

Категориальные данные необходимо преобразовать в числовые для использования в моделях машинного обучения и статистических методов. Существует несколько способов сделать это, каждый из которых имеет свои преимущества в зависимости от структуры данных и требований задачи. Рассмотрим основные методы:

## 1. Преобразование категорий в числовые коды (Label Encoding)

Этот метод заменяет каждую категорию числом. Например, если у нас есть 4 типа груза (`Coal`, `Grain`, `Metal`, `Containers`), они будут заменены числами: `Coal` = 0, `Grain` = 1, `Metal` = 2, и так далее.

##### Пример кодирования категорий:

```python
# Преобразование столбца 'Cargo_Type' в числовой код (Label Encoding)
df['Cargo_Type_Coded'] = df['Cargo_Type'].astype('category').cat.codes

# Вывод первых строк после кодирования
print(df[['Cargo_Type', 'Cargo_Type_Coded']].head())
```

### Пояснение:
- **`astype('category')`** — преобразует столбец в категориальный тип.
- **`cat.codes`** — возвращает числовой код для каждой категории.

### Преимущества:
- Простой и быстрый метод.
- Подходит для категориальных переменных с естественным порядком (например, `low`, `medium`, `high`).

### Недостатки:
- Не подходит для категорий без порядка. Модель может ошибочно воспринять числовые коды как упорядоченные данные.

## 2. Преобразование категорий в дамми-признаки (One-Hot Encoding)

**One-Hot Encoding** — это метод преобразования категориальных данных в несколько бинарных столбцов. Каждый столбец представляет отдельную категорию и содержит значение 1, если наблюдение принадлежит этой категории, или 0 — если нет.

##### Пример One-Hot Encoding:

```python
# Преобразование категориальных данных с использованием дамми-признаков (One-Hot Encoding)
df_one_hot = pd.get_dummies(df, columns=['Cargo_Type'], prefix='Cargo')

# Вывод первых строк после преобразования
print(df_one_hot.head())
```

### Пояснение:
- **`pd.get_dummies()`** — автоматически создает дамми-признаки для выбранных категориальных столбцов.
- **`columns=['Cargo_Type']`** — указывает, что необходимо преобразовать столбец `Cargo_Type`.

### Преимущества:
- Избегает проблемы с упорядочиванием категорий, как в Label Encoding.
- Подходит для категорий без естественного порядка.

### Недостатки:
- Увеличивает количество признаков, что может привести к проблемам с производительностью при большом количестве категорий.
- Может вызвать проблему **"dummy variable trap"** — когда одно значение можно однозначно вычислить по значениям остальных категорий. Обычно это решается удалением одной из категорий.

##### Пример решения dummy variable trap:

```python
# Преобразование с исключением одной категории для избежания dummy variable trap
df_one_hot_trap = pd.get_dummies(df, columns=['Cargo_Type'], drop_first=True)

# Вывод первых строк после преобразования
print(df_one_hot_trap.head())
```

### Пояснение:
- **`drop_first=True`** — исключает первый столбец, чтобы избежать линейной зависимости между признаками.

## 3. Преобразование с использованием Frequency Encoding

**Frequency Encoding** — это метод, при котором каждая категория заменяется частотой её появления в наборе данных. Это полезно, если категориальные переменные сильно варьируются по частоте встречаемости.

##### Пример Frequency Encoding:

```python
# Преобразование категориальных данных с использованием частоты встречаемости (Frequency Encoding)
freq_encoding = df['Cargo_Type'].value_counts(normalize=True)
df['Cargo_Type_Frequency'] = df['Cargo_Type'].map(freq_encoding)

# Вывод первых строк после преобразования
print(df[['Cargo_Type', 'Cargo_Type_Frequency']].head())
```

### Пояснение:
- **`value_counts(normalize=True)`** — вычисляет долю каждого значения в наборе данных.
- **`map()`** — заменяет категории на их частотные значения.

### Преимущества:
- Эффективно для категориальных признаков с большим количеством уникальных значений.
- Не увеличивает размер данных (в отличие от One-Hot Encoding).

### Недостатки:
- Этот метод может вводить корреляции между частотами и зависимой переменной, что не всегда полезно в моделях.

## 4. Преобразование с использованием Target Encoding

**Target Encoding** использует среднее значение целевой переменной для каждой категории. Это полезно в задачах классификации и регрессии, когда нужно учитывать влияние категории на целевую переменную.

##### Пример Target Encoding:

```python
# Пример целевой переменной
df['Delay'] = df['Delay_Hours'] > 5  # Пример бинарной целевой переменной (задержка больше 5 часов)

# Преобразование категориальных данных с использованием Target Encoding
mean_delay = df.groupby('Cargo_Type')['Delay'].mean()
df['Cargo_Type_Target_Encoded'] = df['Cargo_Type'].map(mean_delay)

# Вывод первых строк после преобразования
print(df[['Cargo_Type', 'Cargo_Type_Target_Encoded']].head())
```

### Пояснение:
- **`groupby('Cargo_Type')['Delay'].mean()`** — вычисляет среднее значение целевой переменной для каждой категории.
- **`map()`** — заменяет категории на среднее значение целевой переменной.

### Преимущества:
- Может улучшить качество модели, так как использует информацию о целевой переменной.
- Эффективно для категориальных признаков с большим количеством уникальных значений.

### Недостатки:
- Может привести к переобучению, особенно на маленьких наборах данных.

## Вывод

Каждый метод имеет свои преимущества и недостатки:

1. **Label Encoding** — простой, но может вводить ложный порядок в данных.
2. **One-Hot Encoding** — безопасен для категорий без порядка, но может увеличить размер данных.
3. **Frequency Encoding** — эффективно для признаков с большим количеством уникальных значений, но может ввести корреляции с зависимой переменной.
4. **Target Encoding** — может улучшить модели, но требует осторожности из-за риска переобучения.

Выбор метода зависит от задачи, количества уникальных значений в категориальной переменной и особенностей данных.